In [12]:
from ultralytics import YOLO
import cv2
from ultralytics.utils.plotting import Annotator
import os

In [13]:
version = 'v2'
weight = os.path.join('models', version, 'best.pt')
weight_abspath = os.path.abspath(weight)

print(weight_abspath)

model = YOLO(weight)

c:\Users\sorra\projects\mlproj\gun-detection-system\models\v2\best.pt


In [14]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()

    results = model.predict(frame, conf=0.35, stream=True)

    for result in results:

        # https://stackoverflow.com/questions/75324341/yolov8-get-predicted-bounding-box
        annotator = Annotator(frame)

        boxes = result.boxes
        conf_arr = boxes.conf.numpy()
        conf = 0
        for box in boxes:
            b = box.xyxy[0]
            c = box.cls
            
            if len(conf_arr) > 0:
                conf = conf_arr[0]
            annotator.box_label(b, model.names[int(c)]+" "+f'{conf:.2f}')
    
    frame = annotator.result() 

    cv2.imshow('Gun detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


0: 480x640 (no detections), 203.3ms
Speed: 6.0ms preprocess, 203.3ms inference, 7.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 159.1ms
Speed: 4.0ms preprocess, 159.1ms inference, 14.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 155.5ms
Speed: 0.0ms preprocess, 155.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 157.0ms
Speed: 0.0ms preprocess, 157.0ms inference, 10.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 213.0ms
Speed: 0.0ms preprocess, 213.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 154.8ms
Speed: 0.0ms preprocess, 154.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 159.6ms
Speed: 0.0ms preprocess, 159.6ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 147.8ms
Speed: 1.1ms pre